In [5]:
%pip install torch torchvision torchaudio scikit-image matplotlib

  Using cached torch-2.4.1-cp312-none-macosx_11_0_arm64.whl.metadata (26 kB)
  Using cached torchvision-0.19.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.0 kB)
  Using cached scikit_image-0.24.0-cp312-cp312-macosx_12_0_arm64.whl.metadata (14 kB)
  Using cached matplotlib-3.9.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (11 kB)
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached sympy-1.13.3-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.4.1-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached fsspec-2024.9.0-py3-none-any.whl.metadata (11 kB)
  Using cached setuptools-75.2.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached pillow-11.0.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (9.1 kB)
  Using cached scipy-1.14.1-cp312-cp312-macosx_14_0_arm64.whl.metadata (60 kB)
  Using cached imageio-2.36.0-py3-none-any.w

In [2]:
# Import necessary tools (libraries) to work with data, images, and neural networks
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
from skimage import io
from skimage.transform import resize
import torch.utils.data as data


In [4]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
data_directory = random = os.getenv("DATASET1")
all_files = np.array(os.listdir(data_directory))


if os.path.exists(data_directory):
    print("Directory exists:", data_directory)
    print(os.listdir(data_directory))
else:
    print("Directory does not exist:", data_directory)


Directory exists: /Users/shaanjijoe/MACHINE_LEARNING_ENVIRONMENT/Tuberculosis_dataset/ChinaSet_AllFiles/ChinaSet_AllFiles/CXR_png
['CHNCXR_0492_1.png', 'CHNCXR_0187_0.png', 'CHNCXR_0342_1.png', 'CHNCXR_0286_0.png', 'CHNCXR_0022_0.png', 'CHNCXR_0067_0.png', 'CHNCXR_0572_1.png', 'CHNCXR_0537_1.png', 'CHNCXR_0323_0.png', 'CHNCXR_0636_1.png', 'CHNCXR_0147_0.png', 'CHNCXR_0102_0.png', 'CHNCXR_0417_1.png', 'CHNCXR_0452_1.png', 'CHNCXR_0246_0.png', 'Thumbs.db', 'CHNCXR_0203_0.png', 'CHNCXR_0382_1.png', 'CHNCXR_0083_0.png', 'CHNCXR_0596_1.png', 'CHNCXR_0262_0.png', 'CHNCXR_0227_0.png', 'CHNCXR_0433_1.png', 'CHNCXR_0476_1.png', 'CHNCXR_0163_0.png', 'CHNCXR_0126_0.png', 'CHNCXR_0366_1.png', 'CHNCXR_0657_1.png', 'CHNCXR_0612_1.png', 'CHNCXR_0307_0.png', 'CHNCXR_0556_1.png', 'CHNCXR_0513_1.png', 'CHNCXR_0006_0.png', 'CHNCXR_0043_0.png', 'CHNCXR_0218_0.png', 'CHNCXR_0399_1.png', 'CHNCXR_0119_0.png', 'CHNCXR_0364_1.png', 'CHNCXR_0449_1.png', 'CHNCXR_0004_0.png', 'CHNCXR_0041_0.png', 'CHNCXR_0554_1.p

In [14]:
# Define how we want to prepare our images for the neural network
# This converts the image to a format the network can understand
data_transforms = transforms.Compose([transforms.ToTensor()])


# This function loads and prepares each image for our neural network
def CNNloader(data_root, filename):
    # Load the image and resize it to 224x224 pixels
    filename_actual = data_root + '/' + filename
    data_old = io.imread(filename_actual)
    data_old = resize(data_old, (224, 224))
    data_old = np.array(data_old, dtype=np.float32)
    
    # If the image is black and white, convert it to a color image
    # (Our neural network expects color images)
    if len(data_old.shape) <= 2:
        data_new = np.zeros(data_old.shape + (3,))
        data_new[:,:,0] = data_new[:,:,1] = data_new[:,:,2] = np.array(data_old)
        data_old = np.array(data_new, dtype=np.float32)
    
    # Apply our predefined transformations
    data_old = data_transforms(np.array(data_old))
    return data_old

# This class helps organize our data and labels
class CNNDataLayer(data.Dataset):
    def __init__(self, data_root, filenames, loader):
        self.data_root = data_root
        self.filenames = filenames
        self.loader = loader

    # This tells the computer how to get each image and its label
    def __getitem__(self, index):
        filename = self.filenames[index]
        # We assume the label (0 or 1) is the second-to-last character in the filename
        target = [int(filename[-5])]
        target = torch.from_numpy(np.array(target))
        data = self.loader(self.data_root, filename)
        return data, target

    # This tells the computer how many images we have
    def __len__(self):
        return len(self.filenames)



In [28]:

device = None
if torch.backends.mps.is_available():
    device = torch.device('mps')  # Use Metal Performance Shaders on macOS
else:
    device = torch.device('cpu')  # Fallback to CPU

# print("Using device:", device)
# Get a list of all our image files and shuffle them randomly
all_files = np.array(os.listdir(data_directory))
np.random.shuffle(all_files)

# Split our data: 70% for training, 30% for testing
# train_files = all_files[:round(0.7*all_files.shape[0])]
train_files = all_files[:round(0.01*all_files.shape[0])]
test_files = all_files[round(0.99*all_files.shape[0]):]
print(train_files.shape, test_files.shape)

# Create our datasets using the custom class we defined earlier
data_sets_train = CNNDataLayer(data_root=data_directory, filenames=train_files, loader=CNNloader)
data_sets_test = CNNDataLayer(data_root=data_directory, filenames=test_files, loader=CNNloader)

# Create data loaders that will help feed data to our neural network in batches
data_loaders_train = data.DataLoader(data_sets_train, batch_size=8, shuffle=True, num_workers=0)
data_loaders_test = data.DataLoader(data_sets_test, batch_size=8, shuffle=True, num_workers=0)

# Load a pre-trained neural network (VGG16) that's good at understanding images
model_to_train = models.vgg16(pretrained=True)
# model_to_train.load_state_dict(torch.load("./vgg16-397923af.pth"))

# We don't want to retrain the whole network, so we "freeze" most of it
for param in model_to_train.features.parameters():
    param.require_grad = False

# Modify the last part of the network to focus on our specific task (finding tuberculosis)
num_features = model_to_train.classifier[6].in_features
features = list(model_to_train.classifier.children())[:-1]
features.extend([nn.Linear(num_features, 1), nn.Sigmoid()])
model_to_train.classifier = nn.Sequential(*features)

# Alternatively, we could use a custom-made neural network
# from own_cnn_model import SimpleCNN
# model_to_train = SimpleCNN()
model_to_train = model_to_train.to(device)

# Set up the "loss function" (how the network measures its mistakes)
# and the "optimizer" (how the network learns from its mistakes)
criterion = nn.BCELoss()
optimizer_ft = optim.SGD(model_to_train.parameters(), lr=0.001, momentum=0.9)

# Start the training process
num_epochs = 5  # We'll train for 100 rounds
for epoch in range(1, num_epochs+1):
    print('Epoch is ' + str(epoch))
    train_loss = 0
    test_loss = 0
    total_misclassified_train = total_count_train = 0
    total_misclassified_test = total_count_test = 0
    
    # Training phase
    model_to_train.train()
    for batch_idx, (data_now, target_now) in enumerate(data_loaders_train):
        # Move data to GPU if available
        data_now, target_now = data_now.to(device), target_now.to(device)
        
        # Make predictions
        target_output_model = model_to_train(data_now)
        target_now = target_now.type(torch.FloatTensor).to(device)
        
        # Calculate loss (how wrong the predictions were)
        target_loss = criterion(target_output_model, target_now)
        
        # Calculate how many predictions were wrong
        misclassified_temp = target_output_model[target_now == 1]
        total_misclassified_train += (misclassified_temp < 0.5).sum().item()
        misclassified_temp = target_output_model[target_now == 0]
        total_misclassified_train += (misclassified_temp >= 0.5).sum().item()
        total_count_train += data_now.shape[0]
        
        # Learn from the mistakes (backpropagation)
        optimizer_ft.zero_grad()
        target_loss.backward()
        optimizer_ft.step()
    
    # Testing phase (similar to training, but we don't learn from these)
    model_to_train.eval()
    with torch.no_grad():
        for batch_idx, (data_now, target_now) in enumerate(data_loaders_test):
            data_now, target_now = data_now.to(device), target_now.to(device)
            target_output_model = model_to_train(data_now)
            
            # Calculate how many predictions were wrong
            misclassified_temp = target_output_model[target_now == 1]
            total_misclassified_test += (misclassified_temp < 0.5).sum().item()
            misclassified_temp = target_output_model[target_now == 0]
            total_misclassified_test += (misclassified_temp >= 0.5).sum().item()
            total_count_test += data_now.shape[0]
    
    # Save our progress (the network's current state)
    snapshot_path = './snapshots_trial'
    os.makedirs(snapshot_path, exist_ok=True)
    snapshot_name = f'epoch-{epoch}-trainerror-{total_misclassified_train/total_count_train:.4f}-testerror-{total_misclassified_test/total_count_test:.4f}.pth'
    torch.save(model_to_train.state_dict(), os.path.join(snapshot_path, snapshot_name))



(7,) (7,)
Epoch is 1
Epoch is 2
Epoch is 3
Epoch is 4
Epoch is 5


In [27]:
# Example of how to load a saved model for later use
model_loaded = torch.load("./snapshots/epoch-5-trainerror-0.2857-testerror-0.4286.pth", map_location=lambda storage, loc: storage)
model_to_train.load_state_dict(model_loaded)

/var/folders/bt/f868ty6d66g2scxrj62lpgzc0000gn/T/ipykernel_88117/3568264980.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_loaded = torch.load("./snapshots/epoch-

FileNotFoundError: [Errno 2] No such file or directory: './snapshots/epoch-5-trainerror-0.1447-testerror-0.1558.pth'